This is some connect 

In [296]:
# Must be included at the beginning of each new notebook. Remember to change the app name.
import findspark
findspark.init('/home/ubuntu/spark-3.2.1-bin-hadoop2.7')
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('basics').getOrCreate()

In [297]:
file = 'violence_data_NEW.csv'
df = spark.read.csv(file, header=True, inferSchema=True)

In [298]:
from pyspark.sql.types import (StructField,StringType,IntegerType,StructType)

In [299]:
#3.1 drop the 'Usere feedback' field
columns_to_drop = ['User feedback']
new_df = df.drop(*columns_to_drop)

In [300]:
new_df.columns

['RecordID',
 'Country',
 'Gender',
 'Demographics Question',
 'Demographics Response',
 'Question',
 'Survey Year',
 'Value']

In [301]:
#use your sales average to fill missing data. 
from pyspark.sql.functions import mean
#3.2 Dealing with the null value in 'Value' field
mean_value = df.select(mean(df['Value'])).collect()[0][0]
new_df = new_df.na.fill({'Value': mean_value})

In [302]:
new_df.show()
new_df.printSchema()

+--------+-----------+------+---------------------+---------------------+--------------------+-----------+-----------------+
|RecordID|    Country|Gender|Demographics Question|Demographics Response|            Question|Survey Year|            Value|
+--------+-----------+------+---------------------+---------------------+--------------------+-----------+-----------------+
|       1|Afghanistan|     F|       Marital status|        Never married|... if she burns ...| 01/01/2015|19.76253687315632|
|       1|Afghanistan|     F|            Education|               Higher|... if she burns ...| 01/01/2015|             10.1|
|       1|Afghanistan|     F|            Education|            Secondary|... if she burns ...| 01/01/2015|             13.7|
|       1|Afghanistan|     F|            Education|              Primary|... if she burns ...| 01/01/2015|             13.8|
|       1|Afghanistan|     F|       Marital status| Widowed, divorced...|... if she burns ...| 01/01/2015|             13.8|


In [303]:
new = 'Value'

new_df.select(new).show()
new_df.select(new).describe().show()

+-----------------+
|            Value|
+-----------------+
|19.76253687315632|
|             10.1|
|             13.7|
|             13.8|
|             13.8|
|             17.0|
|             17.3|
|             18.0|
|             18.1|
|             18.2|
|             18.3|
|             18.3|
|             18.8|
|             19.1|
|             20.8|
|19.76253687315632|
|              4.5|
|              4.6|
|              5.2|
|              6.3|
+-----------------+
only showing top 20 rows

+-------+------------------+
|summary|             Value|
+-------+------------------+
|  count|             12600|
|   mean| 19.76253687315705|
| stddev|16.005588776096097|
|    min|               0.0|
|    max|              86.9|
+-------+------------------+



In [304]:
#3.3 Constructing new data from 'Value' called 'Measurement'
from pyspark.sql.functions import mean, col, when 
data_copy_33 = new_df.withColumn(
    'Measurement',
    when(col('Value') == mean_value, 0).otherwise(1)
)


In [305]:
new = 'Measurement'

data_copy_33.select(new).show()
data_copy_33.select(new).describe().show()

+-----------+
|Measurement|
+-----------+
|          0|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          1|
|          0|
|          1|
|          1|
|          1|
|          1|
+-----------+
only showing top 20 rows

+-------+-------------------+
|summary|        Measurement|
+-------+-------------------+
|  count|              12600|
|   mean| 0.8878571428571429|
| stddev|0.31555465373180014|
|    min|                  0|
|    max|                  1|
+-------+-------------------+



In [306]:
data_copy_33.printSchema()

root
 |-- RecordID: integer (nullable = true)
 |-- Country: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Demographics Question: string (nullable = true)
 |-- Demographics Response: string (nullable = true)
 |-- Question: string (nullable = true)
 |-- Survey Year: string (nullable = true)
 |-- Value: double (nullable = false)
 |-- Measurement: integer (nullable = false)



In [307]:
#3.4 Merging with other dataframe
file2 = 'violence_data_merge.csv'
merge_df = spark.read.csv(file2, header=True, inferSchema=True)

In [308]:
merge_df.show()

+--------+----+----+----+----+----+----+----+-------------+
|RecordID| _c1| _c2| _c3| _c4| _c5| _c6| _c7|User feedback|
+--------+----+----+----+----+----+----+----+-------------+
|       1|null|null|null|null|null|null|null|         null|
|       1|null|null|null|null|null|null|null|         null|
|       1|null|null|null|null|null|null|null|         null|
|       1|null|null|null|null|null|null|null|           10|
|       1|null|null|null|null|null|null|null|         null|
|       1|null|null|null|null|null|null|null|         null|
|       1|null|null|null|null|null|null|null|           23|
|       1|null|null|null|null|null|null|null|            e|
|       1|null|null|null|null|null|null|null|           4t|
|       1|null|null|null|null|null|null|null|             |
|       1|null|null|null|null|null|null|null|         null|
|       1|null|null|null|null|null|null|null|         null|
|       1|null|null|null|null|null|null|null|          532|
|       1|null|null|null|null|null|null|

23/10/11 04:12:09 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: RecordID, , , , , , , , User feedback
 Schema: RecordID, _c1, _c2, _c3, _c4, _c5, _c6, _c7, User feedback
Expected: _c1 but found: 
CSV file: file:///home/ubuntu/722_I4/violence_data_merge.csv


In [309]:
merged_data = data_copy_33.join(merge_df, 'RecordID')

In [310]:
merged_data.show()
merged_data.printSchema()

+--------+-----------+------+---------------------+---------------------+--------------------+-----------+-----------------+-----------+----+----+----+----+----+----+----+-------------+
|RecordID|    Country|Gender|Demographics Question|Demographics Response|            Question|Survey Year|            Value|Measurement| _c1| _c2| _c3| _c4| _c5| _c6| _c7|User feedback|
+--------+-----------+------+---------------------+---------------------+--------------------+-----------+-----------------+-----------+----+----+----+----+----+----+----+-------------+
|       1|Afghanistan|     F|       Marital status|        Never married|... if she burns ...| 01/01/2015|19.76253687315632|          0|null|null|null|null|null|null|null|           4e|
|       1|Afghanistan|     F|       Marital status|        Never married|... if she burns ...| 01/01/2015|19.76253687315632|          0|null|null|null|null|null|null|null|         gf43|
|       1|Afghanistan|     F|       Marital status|        Never marri

23/10/11 04:12:09 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: RecordID, , , , , , , , User feedback
 Schema: RecordID, _c1, _c2, _c3, _c4, _c5, _c6, _c7, User feedback
Expected: _c1 but found: 
CSV file: file:///home/ubuntu/722_I4/violence_data_merge.csv


In [311]:
#3.5 Removing all the other fields that do not have anything contains
columns_to_drop = ['_c1', '_c2', '_c3', '_c4', '_c5', '_c6', '_c7']
merged_data = merged_data.drop(*columns_to_drop)

In [312]:
merged_data.printSchema()
new = 'Value'

df.select(new).show()
merged_data.select(new).describe().show()

root
 |-- RecordID: integer (nullable = true)
 |-- Country: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Demographics Question: string (nullable = true)
 |-- Demographics Response: string (nullable = true)
 |-- Question: string (nullable = true)
 |-- Survey Year: string (nullable = true)
 |-- Value: double (nullable = false)
 |-- Measurement: integer (nullable = false)
 |-- User feedback: string (nullable = true)

+-----+
|Value|
+-----+
| null|
| 10.1|
| 13.7|
| 13.8|
| 13.8|
| 17.0|
| 17.3|
| 18.0|
| 18.1|
| 18.2|
| 18.3|
| 18.3|
| 18.8|
| 19.1|
| 20.8|
| null|
|  4.5|
|  4.6|
|  5.2|
|  6.3|
+-----+
only showing top 20 rows

+-------+------------------+
|summary|             Value|
+-------+------------------+
|  count|            378000|
|   mean| 19.76253687315055|
| stddev|16.004974791697258|
|    min|               0.0|
|    max|              86.9|
+-------+------------------+



In [313]:
#4.1 Feature selection & Encoding the string fields
from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.ml.feature import VectorAssembler

In [314]:
#Encoding 'Country'
indexer = StringIndexer(inputCol="Country", outputCol="CountryIndex")
encoder = OneHotEncoder(inputCols=["CountryIndex"], outputCols=["CountryVec"])
df = indexer.fit(df).transform(df)
df = encoder.fit(df).transform(df)

In [315]:
df.printSchema()

root
 |-- RecordID: integer (nullable = true)
 |-- Country: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Demographics Question: string (nullable = true)
 |-- Demographics Response: string (nullable = true)
 |-- Question: string (nullable = true)
 |-- Survey Year: string (nullable = true)
 |-- Value: double (nullable = true)
 |-- User feedback: string (nullable = true)
 |-- CountryIndex: double (nullable = false)
 |-- CountryVec: vector (nullable = true)



In [316]:
#Encoding 'Gender'
indexer = StringIndexer(inputCol="Gender", outputCol="GenderIndex")
encoder = OneHotEncoder(inputCols=["GenderIndex"], outputCols=["GenderVec"])
df = indexer.fit(df).transform(df)
df = encoder.fit(df).transform(df)
df.printSchema()

root
 |-- RecordID: integer (nullable = true)
 |-- Country: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Demographics Question: string (nullable = true)
 |-- Demographics Response: string (nullable = true)
 |-- Question: string (nullable = true)
 |-- Survey Year: string (nullable = true)
 |-- Value: double (nullable = true)
 |-- User feedback: string (nullable = true)
 |-- CountryIndex: double (nullable = false)
 |-- CountryVec: vector (nullable = true)
 |-- GenderIndex: double (nullable = false)
 |-- GenderVec: vector (nullable = true)



In [317]:
#Encoding 'Demographics Response' 
#responseIndexer = StringIndexer(inputCol="Demographics Response", outputCol="ResponseIndex")
#responseEncoder = OneHotEncoder(inputCols=["ResponseIndex"], outputCols=["ResponseVec"])
#df = responseIndexer.fit(df).transform(df)
#df = responseEncoder.fit(df).transform(df)
indexer = StringIndexer(inputCol="Demographics Response", outputCol="ResponseIndex")
df = indexer.fit(df).transform(df)
df.printSchema()

root
 |-- RecordID: integer (nullable = true)
 |-- Country: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Demographics Question: string (nullable = true)
 |-- Demographics Response: string (nullable = true)
 |-- Question: string (nullable = true)
 |-- Survey Year: string (nullable = true)
 |-- Value: double (nullable = true)
 |-- User feedback: string (nullable = true)
 |-- CountryIndex: double (nullable = false)
 |-- CountryVec: vector (nullable = true)
 |-- GenderIndex: double (nullable = false)
 |-- GenderVec: vector (nullable = true)
 |-- ResponseIndex: double (nullable = false)



In [318]:
df.show()
#retreat

new_df = df.na.fill(0)


+--------+-----------+------+---------------------+---------------------+--------------------+-----------+-----+-------------+------------+--------------+-----------+-------------+-------------+
|RecordID|    Country|Gender|Demographics Question|Demographics Response|            Question|Survey Year|Value|User feedback|CountryIndex|    CountryVec|GenderIndex|    GenderVec|ResponseIndex|
+--------+-----------+------+---------------------+---------------------+--------------------+-----------+-----+-------------+------------+--------------+-----------+-------------+-------------+
|       1|Afghanistan|     F|       Marital status|        Never married|... if she burns ...| 01/01/2015| null|         null|         0.0|(69,[0],[1.0])|        0.0|(1,[0],[1.0])|          7.0|
|       1|Afghanistan|     F|            Education|               Higher|... if she burns ...| 01/01/2015| 10.1|         null|         0.0|(69,[0],[1.0])|        0.0|(1,[0],[1.0])|          5.0|
|       1|Afghanistan|   

In [319]:
new_df.show()

+--------+-----------+------+---------------------+---------------------+--------------------+-----------+-----+-------------+------------+--------------+-----------+-------------+-------------+
|RecordID|    Country|Gender|Demographics Question|Demographics Response|            Question|Survey Year|Value|User feedback|CountryIndex|    CountryVec|GenderIndex|    GenderVec|ResponseIndex|
+--------+-----------+------+---------------------+---------------------+--------------------+-----------+-----+-------------+------------+--------------+-----------+-------------+-------------+
|       1|Afghanistan|     F|       Marital status|        Never married|... if she burns ...| 01/01/2015|  0.0|         null|         0.0|(69,[0],[1.0])|        0.0|(1,[0],[1.0])|          7.0|
|       1|Afghanistan|     F|            Education|               Higher|... if she burns ...| 01/01/2015| 10.1|         null|         0.0|(69,[0],[1.0])|        0.0|(1,[0],[1.0])|          5.0|
|       1|Afghanistan|   

In [320]:
from pyspark.sql.functions import skewness
skewness_value = new_df.select(skewness(new_df['Value'])).collect()[0][0]
print(f"Skewness of 'Value': {skewness_value}")


Skewness of 'Value': 1.1445442844796359


In [321]:
from pyspark.sql.functions import sqrt
new_df = new_df.withColumn('new_values', sqrt(new_df['Value']))


In [322]:
new_df.show()
new_df.printSchema()

+--------+-----------+------+---------------------+---------------------+--------------------+-----------+-----+-------------+------------+--------------+-----------+-------------+-------------+------------------+
|RecordID|    Country|Gender|Demographics Question|Demographics Response|            Question|Survey Year|Value|User feedback|CountryIndex|    CountryVec|GenderIndex|    GenderVec|ResponseIndex|        new_values|
+--------+-----------+------+---------------------+---------------------+--------------------+-----------+-----+-------------+------------+--------------+-----------+-------------+-------------+------------------+
|       1|Afghanistan|     F|       Marital status|        Never married|... if she burns ...| 01/01/2015|  0.0|         null|         0.0|(69,[0],[1.0])|        0.0|(1,[0],[1.0])|          7.0|               0.0|
|       1|Afghanistan|     F|            Education|               Higher|... if she burns ...| 01/01/2015| 10.1|         null|         0.0|(69,[

In [323]:
###########################################################

In [324]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [325]:
# The input columns are the feature column names, and the output column is what you'd like the new column to be named. 
assembler = VectorAssembler(
    inputCols=["GenderIndex","RecordID"],
    outputCol="features")

In [326]:
# Now that we've created the assembler variable, let's actually transform the data.
output = assembler.transform(new_df)

In [327]:
# Let's select two columns (the feature and predictor).
# This is now in the appropriate format to be processed by Spark.
final_data = output.select("features",'new_values')
#I tried use 'Value',larger RMSE value
final_data.show()

+---------+------------------+
| features|        new_values|
+---------+------------------+
|[0.0,1.0]|               0.0|
|[0.0,1.0]|3.1780497164141406|
|[0.0,1.0]|3.7013511046643495|
|[0.0,1.0]| 3.714835124201342|
|[0.0,1.0]| 3.714835124201342|
|[0.0,1.0]| 4.123105625617661|
|[0.0,1.0]| 4.159326868617084|
|[0.0,1.0]| 4.242640687119285|
|[0.0,1.0]| 4.254409477236529|
|[0.0,1.0]| 4.266145801540309|
|[0.0,1.0]| 4.277849927241488|
|[0.0,1.0]| 4.277849927241488|
|[0.0,1.0]|  4.33589667773576|
|[0.0,1.0]| 4.370354676682432|
|[0.0,1.0]| 4.560701700396552|
|[1.0,1.0]|               0.0|
|[1.0,1.0]|2.1213203435596424|
|[1.0,1.0]|2.1447610589527217|
|[1.0,1.0]| 2.280350850198276|
|[1.0,1.0]|2.5099800796022267|
+---------+------------------+
only showing top 20 rows



In [328]:
#6.1 Linear Regression
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
#split the data
train_data, test_data = final_data.randomSplit([0.7, 0.3])

lr = LinearRegression( labelCol="new_values")

# Trianing model 
lr_model = lr.fit(train_data)
#print out the result
print(f"Coefficients: {lr_model.coefficients}, Intercept: {lr_model.intercept}")




23/10/11 04:12:11 WARN Instrumentation: [8cff9726] regParam is zero, which might cause numerical instability and overfitting.


Coefficients: [-1.3248450328598067,0.004437415059549771], Intercept: 3.285952392784894


In [329]:

pred_results = lr_model.transform(test_data)

pred_results.show()



+---------+-------------------+------------------+
| features|         new_values|        prediction|
+---------+-------------------+------------------+
|[0.0,1.0]| 3.7013511046643495|3.2903898078444436|
|[0.0,1.0]|  4.123105625617661|3.2903898078444436|
|[0.0,1.0]|  4.242640687119285|3.2903898078444436|
|[0.0,1.0]|  4.266145801540309|3.2903898078444436|
|[0.0,1.0]|  4.277849927241488|3.2903898078444436|
|[0.0,2.0]| 0.6324555320336759|3.2948272229039937|
|[0.0,2.0]| 0.7745966692414834|3.2948272229039937|
|[0.0,2.0]| 0.8944271909999159|3.2948272229039937|
|[0.0,2.0]| 0.8944271909999159|3.2948272229039937|
|[0.0,2.0]| 1.1832159566199232|3.2948272229039937|
|[0.0,3.0]| 0.9486832980505138|3.2992646379635433|
|[0.0,3.0]| 2.6267851073127395|3.2992646379635433|
|[0.0,3.0]| 2.9664793948382653|3.2992646379635433|
|[0.0,3.0]| 3.1304951684997055|3.2992646379635433|
|[0.0,3.0]| 3.1622776601683795|3.2992646379635433|
|[0.0,3.0]| 3.6606010435446255|3.2992646379635433|
|[0.0,4.0]|                0.0|

In [330]:

# Let's evaluate the model against the test data.
test_results = lr_model.evaluate(test_data)

# Interesting results! This shows the difference between the predicted value and the test data.
test_results.residuals.show()

# Let's get some evaluation metrics (as discussed in the previous linear regression notebook).
print("RSME: {}".format(test_results.rootMeanSquaredError))


+--------------------+
|           residuals|
+--------------------+
|  0.4109612968199059|
|   0.832715817773217|
|  0.9522508792748412|
|  0.9757559936958651|
|  0.9874601193970443|
| -2.6623716908703177|
| -2.5202305536625103|
|  -2.400400031904078|
|  -2.400400031904078|
| -2.1116112662840703|
| -2.3505813399130293|
| -0.6724795306508038|
|  -0.332785243125278|
|-0.16876946946383775|
|-0.13698697779516378|
| 0.36133640558108215|
|  -3.303702053023093|
|  -2.987474287006255|
|  -2.755979495517927|
|  -2.755979495517927|
+--------------------+
only showing top 20 rows

RSME: 2.0521394859884947


In [331]:
print("R2: {}".format(test_results.r2))

R2: 0.13483848032663426
